# Crawling Data

## Twitter

In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
def tweet_scrape(tag, count=100):
  list_tweet = []
  for i,tweet in enumerate(sntwitter.TwitterHashtagScraper(tag).get_items()):
    if i > count:
      break
    list_tweet.append({
      'Nama': tweet.user.username, 
      'Content': tweet.content, 
      'Lokasi': tweet.user.location
    })
  return pd.DataFrame(list_tweet) 

In [ ]:
tweet = tweet_scrape('capres2024 AND ganjarpranowo')
tweet.to_csv('ganjarScrape.csv', index=False)

<ipython-input-22-cbc57a5e4014>:8: FutureWarning: content is deprecated, use rawContent instead
  'Content': tweet.content,


## Web

In [ ]:
!pip install requests bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=f317840af9172861690877fc0b2c884305e38860715ff4a23a56d88d308fbe5c
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### pta.trunojoyo.ac.id

In [ ]:
pta = []

for i in range(1,172):
  URL = "https://pta.trunojoyo.ac.id/c_search/byprod/10/" + str(i+1)
  r = requests.get(URL)
  soup_data = BeautifulSoup(r.text, 'html.parser')
  items = soup_data.find('div', {'id':'content_journal'})

  for it in items.findAll('li',{'data-cat':'#luxury'}):
    namaPenulis = it.span.text.strip('\n').replace('Penulis : ', '')
    
    link = it.find('a', 'gray button')['href']
    req = requests.get(link)
    soup_data2 = BeautifulSoup(req.text, 'html.parser')
    abstraks = soup_data2.findAll('div', {'style':'margin: 15px 15px 15px 15px;'})
    abstrak = soup_data2.find('p', {'align':'justify'}).text
    nim = link.replace('https://pta.trunojoyo.ac.id/welcome/detail/', '')

    pta.append([namaPenulis, nim, it.a.text.strip('\n'), abstrak])
    
pta_df = pd.DataFrame(pta, columns=['Penulis', 'NIM', 'Judul', 'Abstrak'])
pta_df

,Penulis,NIM,Judul,Abstrak
0,Adi Chandra Laksono,070411100007,Gerak Pekerja Pada Game Real Time Strategy Men...,Gerak pekerja ada pada game yang memiliki genr...
1,NURRACHMAT,070411100126,RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MEN...,"Perkembangan game yang semakin pesat, memberik..."
2,Muhammad Choirur Rozi,070411100109,EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEA...,Sistem pengenalan wajah adalah suatu sistem un...
3,M Khoiril Anwar,080411100083,IMPLEMENTASI ALGORITMA PRIM DAN DEPTH FIRST ...,Teknologi mobile game beroperating system open...
4,MALIKUL HAMZAH,070411100092,Perancangan Sistem Informasi Badan Kepegawaian...,Kantor Badan Kepegawaian kota Bangkalan adalah...
...,...,...,...,...
848,Rachmad Agung Pambudi,160411100032,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Investasi saham selama ini memiliki resiko ker...
849,Nadila Hidayanti,160411100182,SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...,Information Retrieval (IR) merupakan pengambil...
850,Afni Sakinah,160411100077,KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...,Klasifikasi citra merupakan proses pengelompok...
851,Friska Fatmawatiningrum,160411100084,IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...,Identifikasi atribut pejalan kaki merupakan sa...


In [5]:
pta_df.to_csv('pta.csv', index=False)

### Kompas

In [6]:
class ScrapingKompas:
  def __init__(self, url):
    self.url = url

  def page(self):
    res_ = req.get(self.url)
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='paging__link')[:-2]
    return len(contents_)

  def artikel_page(self, page):
    res_ = req.get(f'{self.url}?page={page}')
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='article__link')
    return contents_

  def get_berita(self):
    berita_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        res = req.get(artikel['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        # ambil data
        judul = soup.find('h1', class_='read__title').string
        isi = isi = ' '.join(soup.find('div', class_='read__content').stripped_strings)
        berita_list.append({
            'Judul': judul,
            'Isi Berita': isi
        })
    self.beritas = berita_list
    return 
  
  def result(self):
    return pd.DataFrame(self.beritas)
  
  def save(self, namefile='beritaKompas'):
    df = pd.DataFrame(self.beritas).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [7]:
data = ScrapingKompas('https://indeks.kompas.com/?site=nasional')

In [9]:
data.result()

AttributeError: ignored

In [ ]:
data.save()

### Tempo

In [ ]:
kompas = []

URL = "https://indeks.kompas.com/?site=nasional"
r = requests.get(URL) 
soup_data = BeautifulSoup(r.text, 'html.parser')
items = soup_data.find('a', 'main-left')

for it in items.findAll('h1','title'):
  judul = it.a.text
  link = it.a['href']

  req = requests.get(link)
    
  soup2 = BeautifulSoup(req.text, 'html.parser')
  contents = soup2.find('div', 'detail-in').text.strip('\nTEMPO.CO,')

  kompas.append([judul, contents])
    
kompas_df = pd.DataFrame(kompas, columns=['Judul', 'Isi Berita'])
kompas_df

In [ ]:
tempo = []

URL = "https://www.tempo.co/indeks/2023-02-21/"
r = requests.get(URL)
  
soup = BeautifulSoup(r.text, 'html.parser')
items = soup.find('main', 'main-left')

for it in items.findAll('h2','title'):
  judul = it.a.text
  link = it.a['href']

  req = requests.get(link)
    
  soup2 = BeautifulSoup(req.text, 'html.parser')
  contents = soup2.find('div', 'detail-in').text.strip('\nTEMPO.CO,')

  tempo.append([judul, contents])
    
tempo_df = pd.DataFrame(tempo, columns=['Judul', 'Isi Berita'])
tempo_df

,Judul,Isi Berita
0,Lika-liku Hidup Wendy Red Velvet Berhasil Gapa...,Jakarta - Tepat hari ini 21 Februari pada 199...
1,9 Hakim MK Dilaporkan atas Dugaan Pemalsuan Pu...,Jakarta - Sembilan hakim Mahkamah Konstitusi ...
2,Menengok Sejarah Hipnoterapi: Berkembang Sejak...,"Jakarta - Hipnosis, atau yang juga dikenal de..."
3,Leeds United Menunjuk Mantan Manajer Watford J...,Jakarta - Leeds United telah menyetujui persy...
4,Hampir 1.000 Orang Terlibat Evakuasi Kapolda J...,Jakarta - Kapolri Jenderal Listyo Sigit Prabo...
...,...,...
285,Prediksi Eintracht Frankfurt vs Napoli di Liga...,Jakarta - Jadwal leg pertama babak 16 besar L...
286,"Penangguhan Penahanan Dikabulkan, Pengemudi Fo...",Jakarta - Polres Metro Jakarta Selatan mengab...
287,"IHSG Akan Menguat Hari Ini, Samuel Sekuritas S...",Jakarta - Tim riset PT Samuel Sekuritas Indon...
288,Mengenang Jeanne Calment yang Tercatat Berumur...,Jakarta - Jeanne Calment perempuan Prancis ya...


In [ ]:
tempo_df.to_csv('berita.csv', index=False)